In [7]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import numpy as np
from numpy import array
import andi 
import pandas as pd

import matplotlib.pyplot as plt
import keras as kr
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [8]:
import sys
sys.path.append('../')
from src.features import Q_measure_2D, Dist_distribution
from src.andi_tools import Andi_to_xy, group_by_length, group_similar_as
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import and prepare the data

In [9]:
# Load the trajectories

# Load the trajectories
df = pd.read_csv("../../../data_2D_task2_bis.csv")
df = df.iloc[:,1:]
all_2D = df.values
all_2D = df.values
list_2D_total = [all_2D[i][~np.isnan(all_2D[i])].reshape(1,-1) for i in np.arange(len(all_2D))]
xdata, ydata = Andi_to_xy(list_2D_total)

FileNotFoundError: [Errno 2] File ../../../data_2D_task2_bis.csv does not exist: '../../../data_2D_task2_bis.csv'

In [10]:
# Load the labels 

df_output = pd.read_csv("../../../ref2.txt", delimiter=";")
D2_output = df_output['1.0'] == 2.0
output_2d= np.array(df_output.loc[(D2_output)]['1.0.1'])
output = [kr.utils.to_categorical(output_2d[i], num_classes=5) for i in range(len(output_2d))]

FileNotFoundError: [Errno 2] File ../../../ref2.txt does not exist: '../../../ref2.txt'

# Prepare the training set and calculate the features

In [5]:
# Calculating the different features 

Q_list, lens = Q_measure_2D(xdata, ydata , 2)
Q_list_2, lens_2 = Q_measure_2D(xdata, ydata , 8)
Dist_list, lens_dist = Dist_distribution(xdata, ydata)

NameError: name 'xdata' is not defined

In [6]:
Q_list.shape

NameError: name 'Q_list' is not defined

In [25]:
# Splitting the data into a training and test set 

fraction = np.int(len(xdata)*0.75)

Q_train, Q_test = Q_list[:fraction], Q_list[fraction:]
Q2_train, Q2_test = Q_list[:fraction], Q_list[fraction:]
Dist_train, Dist_test = Dist_list[:fraction], Dist_list[fraction:]

output_train, output_test = output[:fraction], output[fraction:]

In [100]:
# Calculate the range of lengths of the different features and group them by length.

ran = np.arange(np.min(lens),np.max(lens)+1,1)
ran_2 = np.arange(np.min(lens_2),np.max(lens_2)+1,1)
ran_d = np.arange(np.min(lens_dist),np.max(lens_dist)+1,1)

Q_train_grouped, Q_test_grouped  = group_by_length(Q_train, ran), group_by_length(Q_test, ran)
Q2_train_grouped ,Q2_test_grouped = group_by_length(Q2_train, ran), group_by_length(Q2_test, ran)
Dist_train_grouped, Dist_test_grouped = group_similar_as(Dist_train, Q_train, ran), group_similar_as(Dist_test, Q_test, ran)

output_train_grouped, output_test_grouped = group_similar_as(output_train, Q_train, ran),  group_similar_as(output_test, Q_test, ran)


In [108]:
# Here we just want to remove the empty elements and squeeze the data to prepare the input of the LSTM. 

result, result_test = [x/np.median(x) for x in Q_train_grouped  if x.shape[0]!=0 ], [x/np.median(x) for x in Q_test_grouped  if x.shape[0]!=0 ]
result_2, result2_test  = [x/np.median(x) for x in Q2_train_grouped  if x.shape[0]!=0 ], [x/np.median(x) for x in Q2_test_grouped  if x.shape[0]!=0 ]
result_d, resultd_test = [x for x in Dist_train_grouped if x.shape[0]!=0 ] , [x for x in Dist_test_grouped if x.shape[0]!=0 ]
output_train_grouped, output_test_grouped = [x for x in output_train_grouped if x.shape[0]!=0 ] , [x for x in output_test_grouped if x.shape[0]!=0 ]

In [107]:
 output_test_grouped[1].sh

array([[1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [109]:
result, result_test = [np.squeeze(i,axis=1) for i in result], [np.squeeze(i,axis=1) for i in result_test]
result_2, result2_test = [np.squeeze(i,axis=1) for i in result_2], [np.squeeze(i,axis=1) for i in result2_test]

result_testtion of the model

In [77]:
# define two sets of inputs
inputA = kr.layers.Input(shape=(None,1,))
inputB = kr.layers.Input(shape=(None,1,))
inputC = kr.layers.Input(shape=(None,1,))

In [78]:
# Side A 
x1 =  kr.layers.LSTM(32, activation="sigmoid")(inputA)
x1 =  kr.layers.Dense(32, activation="sigmoid")(x1)
x1 =  kr.Model(inputs=inputA, outputs=x1)

# Side B
x2 = kr.layers.LSTM(32, activation="sigmoid")(inputB)
x2 = kr.layers.Dense(32, activation="sigmoid")(x2)
x2 = kr.Model(inputs=inputB, outputs=x2)

# Side C
x3 = kr.layers.LSTM(32, activation="sigmoid")(inputC)
x3 = kr.layers.Dense(32, activation="sigmoid")(x3)
x3 = kr.Model(inputs=inputC, outputs=x3)

# Combine side A en B
combined = kr.layers.concatenate([x1.output, x2.output, x3.output])
z = kr.layers.Dense(3, activation="sigmoid")(combined)
z = kr.layers.Dense(5, activation="softmax")(z)


In [79]:
model = kr.Model(inputs=[x1.input, x2.input, x3.input], outputs=z)
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])
#print(model.summary())

# Training the model

In [80]:
epochs=10
for j in np.arange(epochs):
    print('epoch:',j, "/", epochs)
    for i in np.arange(0,len(output_grouped)):
        model.train_on_batch([result[i][:,:,None],result_2[i][:,:,None],result_d[i][:,:,None]],  output_train_grouped[i])

epoch: 0 / 10
epoch: 1 / 10
epoch: 2 / 10
epoch: 3 / 10
epoch: 4 / 10
epoch: 5 / 10
epoch: 6 / 10


KeyboardInterrupt: 

In [117]:
epochs=10
for j in np.arange(epochs):
    print('epoch:',j, "/", epochs)
    for i in np.arange(0,len(output_grouped)):
        model.fit([result[i][:,:,None],result_2[i][:,:,None],result_d[i][:,:,None]],  output_train_grouped[i],verbose= 0)

epoch: 0 / 10
epoch: 1 / 10
epoch: 2 / 10
epoch: 3 / 10
epoch: 4 / 10
epoch: 5 / 10


KeyboardInterrupt: 

In [120]:
evaluation = []
for i in np.arange(len(result_test)):
        evaluation.append(np.array(model.evaluate([result_test[i][:,:,None],result2_test[i][:,:,None],resultd_test[i][:,:,None]],  output_test_grouped[i],verbose=False)))

KeyboardInterrupt: 

The evaluation list should give the accurary of the prediction

In [119]:
np.mean(np.array(evaluation)[:,1])

0.3562084975435927